<a href="https://colab.research.google.com/github/JaganMurugan/Learning/blob/main/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import time

"""
print(time.time())

time.sleep(2)

start = time.time()

time.sleep(2)

end = time.time()

print("Elapsed time:", end - start)
"""

'\nprint(time.time())\n\ntime.sleep(2)\n\nstart = time.time()\n\ntime.sleep(2)\n\nend = time.time()\n\nprint("Elapsed time:", end - start)\n'

In [17]:
from typing import Iterable, Tuple
from dataclasses import dataclass

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms

In [18]:
class LeNet5(nn.Module):
  def __init__(self):
    super().__init__()

    self._body = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2)
    )

    self._head = nn.Sequential(
        nn.Linear(in_features=16*5*5, out_features=120),
        nn.ReLU(inplace=True),
        nn.Linear(in_features=120, out_features=84),
        nn.ReLU(inplace=True),
        nn.Linear(in_features=84, out_features=10)
    )

  def forward(self, x):
    x = self._body(x)
    x = x.view(x.size()[0], -1)
    x = self._head(x)

    return x

In [19]:
lenet = LeNet5()
print(lenet)

LeNet5(
  (_body): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (_head): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [20]:
def get_data(batch_size, data_root='data',num_workers=1):

  train_test_transforms = transforms.Compose([
      transforms.Resize(32, 32),
      transforms.ToTensor(),
      transforms.Normalize((0.1307), (0.3081))
  ])

  train_loader = torch.utils.data.DataLoader(
      datasets.MNIST(root=data_root, train=True, download=False, transform=train_test_transforms),
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers
  )

  test_loader = torch.utils.data.DataLoader(
      datasets.MNIST(root=data_root, train=False, download=False, transform=train_test_transforms),
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers
  )

In [21]:
@dataclass
class SystemConfiuration:
  seed: int = 42
  cudnn_benchmark_enabled: bool = True,
  cudnn_determinisitic: bool = True



In [22]:
@dataclass
class TrainingConfiguration:

  batch_size: int = 32

  learning_rate: float = 0.01

In [7]:
import numpy as np

class NearestNeighbor:
  def __init__(self):
    pass

  def train(self, X, y):
    self.Xtr = X
    self.ytr = y

  def predict(self, X):
    num_test = X.shape[0]
    Ypred = np.zeros(num_test, dtype=self.ytr.dtype)

    for i in range(num_test):
      distances = np.sum(np.abs(self.Xtr - X[i,:]), axis=1)
      min_index = np.argmin(distances)
      Ypred[i] = self.ytr[min_index]

    return Ypred

In [8]:
import torch

In [9]:
import torch

In [23]:

def train(
    train_config: TrainingConfiguration, model: nn.Module, optimizer: torch.optim.Optimizer,
    train_loader: torch.utils.data.DataLoader, epoch_idx: int
) -> Tuple[float, float]:
  model.train()

  batch_loss = np.array([])

  batch_acc = np.array([])

  for batch_idx, (data, target) in enumerate(train_loader):
       indx_target = target.clone()

       data = data.to(train_config.device)

       target = target.to(train_config.device)

       optimizer.zero_grad()

       output = model(data)

       loss = F.cross_entropy(output, target)

       loss.backward()

       optimizer.step()

       batch_loss = np.append(batch_loss, [loss.item()])

       prob = F.softmax(output, dim=1)

       pred = prob.data.max(dim=1)[1]

       correct = pred.cpu().eq(indx_target).sum()

       acc = float(correct) / float(len(data))

       batch_acc = np.append(batch_acc, [acc])

       if batch_idx % train_config.log_interval == 0 and batch_idx > 0:
        print(
            'Train Epoch: {} [{}/{}] Loss: {:.6f} Acc: {:.4f}',
            epoch_idx, batch_idx * len(data), len(train_loader.dataset), loss.item(), acc
        )
  epoch_loss = batch_loss.mean()
  epoch_acc = batch_acc.mean()
  return epoch_loss, epoch_acc






In [ ]:
import torch.nn.functional as F

loss = F.cross_entropy(torch.tensor([5.0, 6.0, 7.0]), torch.tensor([5.0, 7.0, 7.0]))

In [ ]:
loss.item()

24.744510650634766

In [ ]:
F.softmax(torch.tensor([[5.0, 6.0, 7.0], [5.0, 6.0, 7.0]]), dim=1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
import torch

In [ ]:
import torch

In [ ]:
import numpy as np

In [ ]:
class NearestNeighbor:
  def __init__(self):
    pass

  def train(self, X, y):
    """ X is N x D where each row is an example. Y is 1-dimension of size N"""
    self.Xtr = X
    self.ytr = y

  def predict(self, X):
    num_test = X.shape[0]

    Ypred = np.zeros(num_test, dtype = self.ytr.dtype)

    for i in range(num_test):
      distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
      min_index = np.argmin(distances)
      Ypred[i] = self.ytr[min_index]

    return Ypred


In [26]:
def validate(
    train_config: TrainingConfiguration,
    model: nn.Module,
    test_loader: torch.utils.data.DataLoader,
) -> Tuple[float, float]:
  model.eval()
  test_loss = 0